<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/ASR01/ASR01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Create Sample sentence using gTTS

## Set-up gTTS
- Make functions: etts(text), ktts(text)

In [ ]:
#@markdown Step #1:🚩 install {Gradio}{gTTS} packages and import:
%%capture
!pip install gradio

import gradio as gr

!pip install gTTS
from gtts import gTTS
from IPython.display import Audio

In [ ]:
#@markdown 🚩 Making functions: etts('text'), ktts('text')
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio

def etts(text):
  text_to_say = text

  gtts_object = gTTS(text = text_to_say,
                     lang = "en",
                    slow = False)
  
  gtts_object.save("E-audio.wav")
  return Audio("E-audio.wav")


def ktts(text):
  text_to_say = text

  gtts_object = gTTS(text = text_to_say,
                     lang = "ko",
                    slow = False)
  
  gtts_object.save("K-audio.wav")
  return Audio("K-audio.wav")
# #@markdown 🚩 Type text to say
# print('Type texts to create audio:')
# txt = input()
# tts(txt)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
etts("sample one")

# [2] Read text (the rainbow passage)
- Processed files (sentence list, wordlist by frequency)

- [sentence list](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentlist.csv) on Github => data1
- [word list](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_wordfreq.csv) on Github => data2

In [ ]:
import pandas as pd
import requests

url_01 = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentlist.csv"
data1 = pd.read_csv(url_01)

url_02 = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_wordfreq.csv"
data2 = pd.read_csv(url_02)

- data1 = rainbow passage sentences (18)
- data2 = rainbow passage words (331) by frequency

## 🅾️  Pop-up to select a sentence for audio generation

## 1) Generating English accented audio

In [ ]:
#@markdown 🚩 Create audio with English accent for selected sentence
#@markdown Audio file will be saved as, e.g., "ES01.wav"
# !pip install gTTS
# from gtts import gTTS
# from IPython.display import Audio

sn = input('Which sentence? to audio? (1~18)')
sn = int(sn) - 1
txt = data1['Sentence'][sn]
print('S%d: %s'%((sn+1), txt))
filename = "ES" + str(sn+1)+".wav"

def tts(text):
  text_to_say = text
  gtts_object = gTTS(text = text_to_say,
                     lang = "en",
                    slow = False)
  gtts_object.save(filename)
  return Audio(filename)

tts(txt)


Which sentence? to audio? (1~18)1
S1: When the sunlight strikes raindrops in the air, they act as a prism and form a rainbow.


## 2) Generating Korean accented audio

In [ ]:
#@markdown 🚩 Create audio with Korean accent for selected sentence
#@markdown Audio file will be saved as, e.g., "KS01.wav"

sn = input('Which sentence? to audio? (1~18)')
sn = int(sn) - 1
txt = data1['Sentence'][sn]
print('S%d: %s'%((sn+1), txt))
filename = "KS" + str(sn+1)+".wav"

def tts(text):
  text_to_say = text
  gtts_object = gTTS(text = text_to_say,
                     lang = "ko",
                    slow = False)
  gtts_object.save(filename)
  return Audio(filename)

tts(txt)


Which sentence? to audio? (1~18)2
S2: The rainbow is a division of white light into many beautiful colors.


# Sample audio files from github

- man_lainbow.wav: manipulated audio (/r/ is replaced with /l/ in 'rainbow')
- ES02.wav: no manipulation (original gTTS voice of the sample sentence)

sample text: "The rainbow is a division of white light into many beautiful colors."

## Demo: gTTS speech vs. manipulated speech (an error is added)

In [ ]:
import requests
from IPython.display import Audio

In [ ]:
url= "https://github.com/MK316/workspace/raw/main/ASR01/sounds/man_lainbow.wav"
response = requests.get(url)

with open('man_lainbow.wav', 'wb') as f:
  f.write(response.content)

Audio(url)

# Speech recognition text

In [ ]:
%%capture
!pip install git+https://github.com/openai/whisper.git 

In [ ]:
import whisper

model = whisper.load_model('base.en')
# result = model.transcribe("man_lainbow.wav", language="en",fp16=False)
# print(result["text"])   

100%|███████████████████████████████████████| 139M/139M [00:06<00:00, 20.9MiB/s]


In [ ]:
#@markdown Speech-to-text (you need to type the audio file name - see the left panel.)

#@markdown - If you want to upload a file directly, use the code below, and then run this code.
myfile = input("Type file name: e.g., ES1.wav")
filepath = "/content/"+ myfile
result = model.transcribe(myfile, language="en",fp16=False)
print(result["text"])
Audio(filepath)  

Type file name: e.g., ES1.wavman_lainbow.wav
 The Lain Bowl is a division of white light into many beautiful colors.


In [ ]:
## File to upload (if necessary)
# from google.colab import files
# uploaded = files.upload()

# Error rate~Perception rate

Percent of the number of words matching to the original text

### Demo

- Sample text: "**The rainbow** is a division of white light into many beautiful colors." (12 words)

- Perceived text: **The lane bowl** is a division of white light into many beautiful colors. (11 words)

Perception rate = (11/12)*100 = 91%  
Error rate = 9%

- Expected text: S1
- Comparing text: S1 (1 error added) = man_lainbow.wav


In [ ]:
#@markdown Calculate recognition rate: 
from nltk.tokenize import RegexpTokenizer

sample = data1['Sentence'][1]

# text 1
txt1 = sample.lower()
tokenizer = RegexpTokenizer(r'\w+')
wlist = tokenizer.tokenize(txt1)
wlist

# text 2
comptxt = result["text"]
txt2 = comptxt.lower()
tokenizer = RegexpTokenizer(r'\w+')
wlist1 = tokenizer.tokenize(txt2)
wlist1

# Recognition rate

from tables.idxutils import calc_chunksize
from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)

mword = []
score = 0
for i in range(0, len(wlist1)):
  w = wlist1[i]
  if w in wlist:
    sc = 1
  else:
    sc = 0
  score = score + sc

missingword = (score/len(wlist))*100
# RecRate = round(missingword,2)
RecRate = format(round(missingword,2), '.2f')
# ErrRate = round((100.0 - RecRate),2)
ErrRate = format(round((100.0-RecRate),2), '.2f')
print('Matching words: %d'%score, 'out of %d words'%len(wlist))
print('Recognition Rate: %f %%'%RecRate)
print('Error Rate: %f %%'%ErrRate)

Matching words: 11 out of 12 words
Recognition Rate: 91.670000 %
Error Rate: 8.330000 %


rounding and format (Remove 0 trails in 3.333000)

In [4]:
# a = (11/12)*100
# a1 = format(a,'.3f')
# b = 100-a
# b1 = format(b, '.2f'); b1

'8.33'

Checked upto Here (12/29)

### Find matching words: output is the index

In [ ]:
def matchword(word):
  result = txt2.find(word)
  if result == -1:
    print("None")
  else:
    print(result)

In [ ]:
matchword("white")

26


# Sample audio files from github

In [ ]:
import requests
from IPython.display import Audio

In [ ]:
url= "https://github.com/MK316/workspace/raw/main/ASR01/sounds/man_lainbow.wav"
response = requests.get(url)

with open('man_lainbow.wav', 'wb') as f:
  f.write(response.content)

Audio(url)

In [ ]:
url = "https://github.com/MK316/workspace/raw/main/ASR01/sounds/ES02.wav"
with open('ES02.wav', 'wb') as f:
  f.write(response.content)
Audio(url)

Audio file from github and save/load it

In [ ]:
import requests
from IPython.display import Audio

# Set the URL of the audio file
url = "https://github.com/MK316/workspace/raw/main/ASR01/sounds/man_lainbow.wav"

# Download the audio file
response = requests.get(url)

with open('man_lainbow.wav', 'wb') as f:
  f.write(response.content)
# Load the audio file into memory
audio = response.content

# Play the audio
Audio(audio)